## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [201]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [202]:
# load the data - it is available open source and online

data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

# display data
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [203]:
# replace interrogation marks by NaN values

data = data.replace('?', np.nan)

In [204]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan
    
data['cabin'] = data['cabin'].apply(get_first_cabin)

In [205]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'
    
data['title'] = data['name'].apply(get_title)

In [206]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [207]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs


In [208]:
# save the data set
data.to_csv('./Data/titanic-01.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [209]:
target = 'survived'

In [210]:
vars_num = data.select_dtypes(include='number')
vars_num = vars_num[[col for col in vars_num.columns if col != 'survived']]

vars_cat = data.select_dtypes(include='object')

print('Number of numerical variables: {}'.format(len(vars_num.columns)))
print('Number of categorical variables: {}'.format(len(vars_cat.columns)))

Number of numerical variables: 5
Number of categorical variables: 4


### Find missing values in variables

In [211]:
vars_num.isnull().sum()

pclass      0
age       263
sibsp       0
parch       0
fare        1
dtype: int64

In [212]:
vars_cat.isnull().sum()

sex            0
cabin       1014
embarked       2
title          0
dtype: int64

### Determine cardinality of categorical variables

In [213]:
vars_cat.nunique()

sex           2
cabin       181
embarked      3
title         5
dtype: int64

### Determine the distribution of numerical variables

In [214]:
vars_num.nunique()

pclass      3
age        98
sibsp       7
parch       8
fare      281
dtype: int64

## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [215]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1047, 9), (262, 9))

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [216]:
X_train.cabin = X_train.cabin.str.replace(r'\d+', '', regex = True)

X_train.cabin.unique()

array([nan, 'E', 'F', 'A', 'C', 'D', 'B', 'T', 'G'], dtype=object)

### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [217]:
for col in vars_num:
    X_train[col + "_missing"] = X_train[col].isnull()
    X_train[col] = X_train[col].fillna(X_train[col].median())

    X_test[col + "_missing"] = X_test[col].isnull()
    X_test[col] = X_test[col].fillna(X_train[col].median())

X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_missing,age_missing,sibsp_missing,parch_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,NaN,S,Mr,False,False,False,False,False
44,1,female,41.0000,0,0,134.5000,E,C,Miss,False,False,False,False,False
1072,3,male,28.0000,0,0,7.7333,NaN,Q,Mr,False,True,False,False,False
1130,3,female,18.0000,0,0,7.7750,NaN,S,Miss,False,False,False,False,False
574,2,male,29.0000,1,0,21.0000,NaN,S,Mr,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,NaN,S,Miss,False,False,False,False,False
835,3,male,28.0000,0,0,8.0500,NaN,S,Mr,False,True,False,False,False
1216,3,female,28.0000,0,0,7.7333,NaN,Q,Miss,False,True,False,False,False
559,2,female,20.0000,0,0,36.7500,NaN,S,Miss,False,False,False,False,False


### Replace Missing data in categorical variables with the string **Missing**

In [218]:
X_train = X_train.fillna("Missing")
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_missing,age_missing,sibsp_missing,parch_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,Missing,S,Mr,False,False,False,False,False
44,1,female,41.0000,0,0,134.5000,E,C,Miss,False,False,False,False,False
1072,3,male,28.0000,0,0,7.7333,Missing,Q,Mr,False,True,False,False,False
1130,3,female,18.0000,0,0,7.7750,Missing,S,Miss,False,False,False,False,False
574,2,male,29.0000,1,0,21.0000,Missing,S,Mr,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,Missing,S,Miss,False,False,False,False,False
835,3,male,28.0000,0,0,8.0500,Missing,S,Mr,False,True,False,False,False
1216,3,female,28.0000,0,0,7.7333,Missing,Q,Miss,False,True,False,False,False
559,2,female,20.0000,0,0,36.7500,Missing,S,Miss,False,False,False,False,False


In [219]:
X_test = X_test.fillna("Missing")
X_test

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_missing,age_missing,sibsp_missing,parch_missing,fare_missing
1139,3,male,38.0,0,0,7.8958,Missing,S,Mr,False,False,False,False,False
533,2,female,21.0,0,1,21.0000,Missing,S,Miss,False,False,False,False,False
459,2,male,42.0,1,0,27.0000,Missing,S,Mr,False,False,False,False,False
1150,3,male,28.0,0,0,14.5000,Missing,S,Mr,False,True,False,False,False
393,2,male,25.0,0,0,31.5000,Missing,S,Mr,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,3,male,22.0,0,0,8.0500,Missing,S,Mr,False,False,False,False,False
1052,3,male,28.0,0,0,7.8958,Missing,S,Mr,False,True,False,False,False
426,2,male,30.0,0,0,13.0000,Missing,S,Mr,False,False,False,False,False
554,2,male,26.0,0,0,13.0000,Missing,S,Mr,False,False,False,False,False


### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [220]:
threshold = 0.05
for col in vars_cat:
    counts = X_train[col].value_counts(normalize=True)
    X_train[col] = np.where(X_train[col].isin(counts[counts > threshold].index), X_train[col], "Rare")
    X_test[col] = np.where(X_test[col].isin(counts[counts > threshold].index), X_test[col], "Rare")

X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_missing,age_missing,sibsp_missing,parch_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,Missing,S,Mr,False,False,False,False,False
44,1,female,41.0000,0,0,134.5000,Rare,C,Miss,False,False,False,False,False
1072,3,male,28.0000,0,0,7.7333,Missing,Q,Mr,False,True,False,False,False
1130,3,female,18.0000,0,0,7.7750,Missing,S,Miss,False,False,False,False,False
574,2,male,29.0000,1,0,21.0000,Missing,S,Mr,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,Missing,S,Miss,False,False,False,False,False
835,3,male,28.0000,0,0,8.0500,Missing,S,Mr,False,True,False,False,False
1216,3,female,28.0000,0,0,7.7333,Missing,Q,Miss,False,True,False,False,False
559,2,female,20.0000,0,0,36.7500,Missing,S,Miss,False,False,False,False,False


### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [221]:
X_train = pd.get_dummies(X_train, vars_cat.columns)

X_train

,pclass,age,sibsp,parch,fare,pclass_missing,age_missing,sibsp_missing,parch_missing,fare_missing,sex_female,sex_male,cabin_C,cabin_Missing,cabin_Rare,embarked_C,embarked_Q,embarked_Rare,embarked_S,title_Miss,title_Mr,title_Mrs,title_Rare
1118,3,25.0000,0,0,7.9250,False,False,False,False,False,0,1,0,1,0,0,0,0,1,0,1,0,0
44,1,41.0000,0,0,134.5000,False,False,False,False,False,1,0,0,0,1,1,0,0,0,1,0,0,0
1072,3,28.0000,0,0,7.7333,False,True,False,False,False,0,1,0,1,0,0,1,0,0,0,1,0,0
1130,3,18.0000,0,0,7.7750,False,False,False,False,False,1,0,0,1,0,0,0,0,1,1,0,0,0
574,2,29.0000,1,0,21.0000,False,False,False,False,False,0,1,0,1,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,3,0.1667,1,2,20.5750,False,False,False,False,False,1,0,0,1,0,0,0,0,1,1,0,0,0
835,3,28.0000,0,0,8.0500,False,True,False,False,False,0,1,0,1,0,0,0,0,1,0,1,0,0
1216,3,28.0000,0,0,7.7333,False,True,False,False,False,1,0,0,1,0,0,1,0,0,1,0,0,0
559,2,20.0000,0,0,36.7500,False,False,False,False,False,1,0,0,1,0,0,0,0,1,1,0,0,0


In [223]:
X_test = pd.get_dummies(X_test, vars_cat.columns)

X_test

,pclass,age,sibsp,parch,fare,pclass_missing,age_missing,sibsp_missing,parch_missing,fare_missing,sex_female,sex_male,cabin_Missing,cabin_Rare,embarked_C,embarked_Q,embarked_S,title_Miss,title_Mr,title_Mrs,title_Rare
1139,3,38.0,0,0,7.8958,False,False,False,False,False,0,1,1,0,0,0,1,0,1,0,0
533,2,21.0,0,1,21.0000,False,False,False,False,False,1,0,1,0,0,0,1,1,0,0,0
459,2,42.0,1,0,27.0000,False,False,False,False,False,0,1,1,0,0,0,1,0,1,0,0
1150,3,28.0,0,0,14.5000,False,True,False,False,False,0,1,1,0,0,0,1,0,1,0,0
393,2,25.0,0,0,31.5000,False,False,False,False,False,0,1,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,3,22.0,0,0,8.0500,False,False,False,False,False,0,1,1,0,0,0,1,0,1,0,0
1052,3,28.0,0,0,7.8958,False,True,False,False,False,0,1,1,0,0,0,1,0,1,0,0
426,2,30.0,0,0,13.0000,False,False,False,False,False,0,1,1,0,0,0,1,0,1,0,0
554,2,26.0,0,0,13.0000,False,False,False,False,False,0,1,1,0,0,0,1,0,1,0,0


In [228]:
X_test["embarked_Rare"] = 0
X_test["cabin_C"] = 0

### Scale the variables

- Use the standard scaler from Scikit-learn

In [229]:
stdScale = StandardScaler()

stdScale.fit(X_train)

X_train = stdScale.transform(X_train)
X_test = stdScale.transform(X_test)

## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [231]:
model = LogisticRegression(random_state=0, C = 0.0005).fit(X_train, y_train)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [241]:
pred_class_train = model.predict(X_train)
pred_prob_train = model.predict_proba(X_train)[:, 1]

print("Train ROC Curve: {}".format(roc_auc_score(y_train, pred_prob_train)))
print("Train Accuracy: {}".format(accuracy_score(y_train, pred_class_train)))

pred_class_test = model.predict(X_test)
pred_prob_test = model.predict_proba(X_test)[:, 1]

print("Train ROC Curve: {}".format(roc_auc_score(y_test, pred_prob_test)))
print("Train Accuracy: {}".format(accuracy_score(y_test, pred_class_test)))

Train ROC Curve: 0.844242658423493
Train Accuracy: 0.7344794651384909
Train ROC Curve: 0.7478086419753086
Train Accuracy: 0.6717557251908397


That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**